fcn那个就不push了， 没啥意义，基本和doc一样。<br>
这个是那个deeplab.<br>
数据按网上说的方式存（默认好像是该文件夹） <br>
这个也是拼了几个doc拼出来的，我基本没写几句。<br>
要说一句，我一般是很爱写comments的人。只是这玩意儿我不太懂，我也写不出啥来。<br>
还有我微信里说错了，我电脑慢（还说数据太多了），我好像连一个epoch都没有跑完，只是一个epoch里跑了无数个batch. <br>

In [1]:
import time
import numpy as np
import mxnet as mx
from mxnet import gluon, autograd
from mxnet.gluon.data.vision import transforms
from gluoncv import data, utils, model_zoo
from gluoncv.utils.parallel import *
from gluoncv.loss import MixSoftmaxCrossEntropyLoss
from matplotlib import pyplot as plt

In [2]:
input_transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize([.485, .456, .406], [.229, .224, .225]),])
trainset = data.VOCSegmentation(split='train', transform=input_transform)
print('Training images:', len(trainset))

Training images: 2913


In [3]:
batch_size = 5 #原来好像是2，是不是太少了,可是5好像也很少。
train_data = gluon.data.DataLoader(trainset, batch_size, shuffle=True, last_batch='rollover', num_workers=batch_size)

In [4]:
model = model_zoo.get_model('deeplab_resnet101_ade', pretrained=True)

criterion = MixSoftmaxCrossEntropyLoss(aux=True)
lr_scheduler =utils.LRScheduler('poly', base_lr=0.001,nepochs=50, iters_per_epoch=len(train_data), power=0.9)

ctx_list = [mx.cpu(0)]
model = DataParallelModel(model, ctx_list)
criterion = DataParallelCriterion(criterion, ctx_list)

kv = mx.kv.create('device')
optimizer = gluon.Trainer(model.module.collect_params(), 'sgd', {'lr_scheduler': lr_scheduler,'wd':0.0001, 'momentum': 0.9,
                           'multi_precision': True},
                          kvstore = kv)

In [8]:
train_loss = 0.0
epoch = 0

while epoch < 10:
    epoch += 1
    for batch, (data, target) in enumerate(train_data):
        batch_time = time.time()
        with autograd.record(True):
            outputs = model(data)
            losses = criterion(outputs, target)
            mx.nd.waitall()
            autograd.backward(losses)
        optimizer.step(batch_size)
        for loss in losses:
            train_loss += loss.asnumpy()[0] / len(losses)
        batch_time = time.time() - batch_time
        print(f'Epoch {epoch}, batch {batch}, training loss {round(train_loss/(batch+1),3)}, took {batch_time} seconds')
 


Epoch 1, batch 0, training loss 4.774, took 59.083884954452515 seconds
Epoch 1, batch 1, training loss 2.46, took 71.3262734413147 seconds
Epoch 1, batch 2, training loss 2.238, took 67.65197086334229 seconds


KeyboardInterrupt: 

In [1]:
# add some util function
VOC_COLORMAP = [[0, 0, 0], [128, 0, 0], [0, 128, 0], [128, 128, 0],
                [0, 0, 128], [128, 0, 128], [0, 128, 128], [128, 128, 128],
                [64, 0, 0], [192, 0, 0], [64, 128, 0], [192, 128, 0],
                [64, 0, 128], [192, 0, 128], [64, 128, 128], [192, 128, 128],
                [0, 64, 0], [128, 64, 0], [0, 192, 0], [128, 192, 0],
                [0, 64, 128]]
# Save to the d2l package. 
VOC_CLASSES = ['background', 'aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair', 'cow',
               'diningtable', 'dog', 'horse', 'motorbike', 'person',
               'potted plant', 'sheep', 'sofa', 'train', 'tv/monitor']

In [2]:
def build_colormap2label():
    """Build a RGB color to label mapping for segmentation."""
    colormap2label = nd.zeros(256 ** 3)
    for i, colormap in enumerate(VOC_COLORMAP):
        colormap2label[(colormap[0]*256 + colormap[1])*256 + colormap[2]] = i
    return colormap2label

def voc_label_indices(colormap, colormap2label):
    """Map a RGB color to a label."""
    colormap = colormap.astype('int32')
    idx = ((colormap[:, :, 0] * 256 + colormap[:, :, 1]) * 256
           + colormap[:, :, 2])
    return colormap2label[idx]